# Compute the Tropical Nights Climate Index

This notebook computes the **Tropical Nights index**: starting from the daily minimum temperature TN,
the Tropical Nights index is the number of days where $TN > T$ (T is  a reference temperature, e.g. 20°C)

As first step, let's connect to the Ophidia server instance

In [ ]:
from os.path import expanduser
home = expanduser("~")
from PyOphidia import cube
cube.Cube.setclient(read_env=True)

Import the input NetCDF data set (with minimum temperature in °K)

In [ ]:
mintemp = cube.Cube.importnc2(src_path="/public/data/tasmin_day_CMCC-ESM2_ssp585_r1i1p1f1_gn_20900101-21001231.nc",
    measure='tasmin',
    #import_metadata='yes',
    imp_dim='time',
    imp_concept_level='d', vocabulary='CF',hierarchy='oph_base|oph_base|oph_time',
    ncores=4,
    nfrag=4,
    description='Daily Min Temp'
)

Identify the tropical nights: $\{day \mid TN(day) > 293.15\}$  with apply (we are basically creating a mask)

In [ ]:
tropicalnights = mintemp.apply(
    query="oph_predicate('OPH_FLOAT','OPH_INT',measure,'x-293.15','>0','1','0')"
)

Count the number of yearly tropical nights

In [ ]:
count = tropicalnights.reduce2(
    operation='sum',
    dim='time',
    concept_level='y'
)

Export the result to a NetCDF file

In [ ]:
count.exportnc2(
    output_path=home,
    output_name="Tropical_Nights_2090-2100"
)